In [130]:
import pyabc
import scipy
import numpy as np
from pyabc.visualization import plot_kde_matrix_highlevel,plot_kde_matrix2
from datetime import datetime
import pandas as pd
from arviz import hdi
from pyabc.visualization import plot_kde_matrix_highlevel, plot_kde_matrix
import numpy as np
import matplotlib.pyplot as plt

path = '/Users/adsurbum/sim-data/single-model-abc/new4'
output_path = '../../figures'

f_no_aneuploidy = '2021-12-29-no-aneuploidy'
f_with_aneuploidy = '2021-12-29-an200-k1'
f100 = '2022-01-04-tau1-r100-22'
f_wider_range = '2022-02-13-wider-mutation-rate-r100'
f_fixedm5 = '2022-03-09-m5-r1000' 
f_fixedm6 = '2022-02-20-m6-r1000'
f_fixedm7 = '2022-03-09-m7-r1000'
f_fixedm8 = '2022-03-09-m8-r1000'
f_prior4 = '2022-02-26-prior4-r1000' # 0.370941 epsilon #bug it is not 1000 reps, it is 100, rerun
# f_prior4 = '2022-01-24-prior4-tau1-1kreps'

In [128]:
f = f_prior4
history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path,f))
history.get_all_populations()[-2:]

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [89]:
k10 = '2021-12-20-an200-k10'
k5 = '2021-12-20-an200-k5' 
k2 = '2021-12-20-an200-k2' 
k1 = '2021-12-29-an200-k1'
k103125 = '2021-12-16-an200-1000reps22'
k100 = '2022-01-12-tau100-r1k'
allf = [k1, k103125, k2, k5, k10, k100]

In [162]:
from scipy.special import logsumexp

def WAIC_with_weights(likelihoods, rounding=None, weights = None):
    if not weights:
        weights = [1/len(likelihoods)]*len(likelihoods)
    loglik_E = np.log(sum(a*b for a,b in zip(weights,likelihoods)))
    #for p2
    E_loglik = sum(a*np.log(b) for a,b in zip(weights,likelihoods))
    p2 = sum((w*(l-E_loglik)**2 for w,l in zip(weights,np.log(likelihoods))))
    ans =  -2*loglik_E + 2*p2
    if rounding:
        ans = round(ans, rounding)
    return ans

def WAIC_for_weighted_samples(f, tminus=0):
    # TODO maybe it should be calculated using kde samples...
    history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path,f))
    hist_stats = history.get_weighted_sum_stats(t=history.max_t-tminus)
    likelihoods = [a['res']-1 for a in hist_stats[1]]
    likelihoods = -np.array(likelihoods) + 0.000001
    weights = hist_stats[0]
    return WAIC_with_weights(likelihoods, weights)

# just another implementation for sanity check
def WAIC_version2(logliks, ddof=0): 
    l_logliks = [np.log(l) for l in logliks]
    l_logliks = np.array(l_logliks)
    S = len(l_logliks)
    llpd = -np.log(S) + logsumexp(l_logliks)
#     p1 = 2*(-np.log(S) + logsumexp(l_logliks) - l_logliks.mean())
    p2 = np.var(l_logliks, ddof=ddof)
    return -2*(llpd + -p2)

# bootstraping WAIC
def bootstrap(likelihoods, roundd=1, n=1000, sampling_size=1000):
    n = 1000
    WAICs = []
    for _ in range(n):
        choice = np.random.choice(likelihoods, size = sampling_size, replace=True)
        WAICs.append(WAIC_with_weights(choice))
    alpha=0.95
    ordered = np.sort(np.array(WAICs))
    lower = np.percentile(ordered, 100*(1-alpha)/2)
    upper = np.percentile(ordered, 100*(alpha+((1-alpha)/2)))
    return (round(np.median(ordered),roundd),round(lower,roundd),round(upper,roundd))

def getl(arr, add=0):
    return [a[1]+add for a in arr] 

In [160]:
class W_RV(pyabc.RVBase):
    def __init__(self):
        l = np.load('../../data/evo39_fitness_39deg.npz')
        self.kde = scipy.stats.gaussian_kde(l['arr_0'])
    def rvs(self, *args, **kwargs):
        return self.kde.resample(1)[0][0]
    def pdf(self, x, *args, **kwargs):
        return self.kde.pdf(x)[0]
    def copy(self):
        raise NotImplementedError('copy')
    def pmf(self, x, *args, **kwargs):
        raise NotImplementedError('pmf')
    def cdf(self, x, *args, **kwargs):
        raise NotImplementedError('cdf')
        
class D_RV(pyabc.RVBase):
    def __init__(self):
        l = np.load('../../data/refined_vs_evo39_fitness_39deg.npz')
        self.kde = scipy.stats.gaussian_kde(l['arr_0'])
    def rvs(self, *args, **kwargs):
        return self.kde.resample(1)[0][0]
    def pdf(self, x, *args, **kwargs):
        return self.kde.pdf(x)[0]
    def copy(self):
        raise NotImplementedError('copy')
    def pmf(self, x, *args, **kwargs):
        raise NotImplementedError('pmf')
    def cdf(self, x, *args, **kwargs):
        raise NotImplementedError('cdf')
        
prior = pyabc.Distribution(p1_mr=pyabc.RV("uniform", 10.0**-9, 10.0**-5-10.0**-9)
                            ,p2_tr=pyabc.RV("uniform", 10.0**-6, 10.0**-2-10.0**-6)
                            ,p3_w1=W_RV()
                            ,p4_w2=W_RV()
                            ,p5_w3=W_RV())

prior_alt = pyabc.Distribution(p1_mr=pyabc.RV("uniform", 10.0**-9, 10.0**-5-10.0**-9)
                            ,p2_tr=pyabc.RV("uniform", 10.0**-6, 10.0**-2-10.0**-6)
                            ,p3_w1=W_RV()
                            ,p4_w2=D_RV()
                            ,p5_w3=D_RV())

prior_fixed_mr = pyabc.Distribution(p2_tr=pyabc.RV("uniform", 10.0**-6, 10.0**-2-10.0**-6)
                            ,p3_w1=W_RV()
                            ,p4_w2=W_RV()
                            ,p5_w3=W_RV())

prior_no_aneuploidy = pyabc.Distribution(p1_mr=pyabc.RV("uniform", 10.0**-9, 10.0**-5-10.0**-9)
                        ,p5_w3=W_RV())

def WAIC_for(f, fixed_mut_rate=False, no_aneuploidy=False, alt_prior=False):
    history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path, f))
    samples, weights = history.get_distribution(m=0, t=None)
    columns = list(samples.columns)
    kde = scipy.stats.gaussian_kde(samples.T.values.tolist(), weights=weights)
    size = 50000
    samples = kde.resample(size)
    posteriors = kde.pdf(samples)
    if fixed_mut_rate:
        priors = [prior_fixed_mr.pdf(dict(zip(['p2_tr','p3_w1','p4_w2','p5_w3'],s)))+1e-6 for s in samples.T]
    elif no_aneuploidy:
        priors = [prior_no_aneuploidy.pdf(dict(zip(['p1_mr','p5_w3'],s)))+1e-6 for s in samples.T]
    elif alt_prior:
        priors = [prior_alt.pdf(dict(zip(['p1_mr','p2_tr','p3_w1','p4_w2','p5_w3'],s)))+1e-6 for s in samples.T]
    else:
        priors = [prior.pdf(dict(zip(['p1_mr','p2_tr','p3_w1','p4_w2','p5_w3'],s)))+1e-6 for s in samples.T]
    likelihoods = posteriors/priors 
    return round(WAIC_with_weights(likelihoods))

In [115]:
results = []
results.append(dict(Model='Without aneuploidy', WAIC=WAIC_for(f_no_aneuploidy, fixed_mut_rate=False, no_aneuploidy=True)))
results.append(dict(Model='Fixed mutation rate, $\mu=10^{{-{5}}}$, $\\tau=1$', WAIC=WAIC_for(f_fixedm5, fixed_mut_rate=True)))
results.append(dict(Model='Fixed mutation rate, $\mu=10^{{-{6}}}$, $\\tau=1$', WAIC=WAIC_for(f_fixedm6, fixed_mut_rate=True)))
results.append(dict(Model='Fixed mutation rate, $\mu=10^{{-{7}}}$, $\\tau=1$', WAIC=WAIC_for(f_fixedm7, fixed_mut_rate=True)))
labels = ['Free mutation rate, $\\tau=1$','Free mutation rate, $\\tau=33/32$',
          'Free mutation rate, $\\tau=2$','Free mutation rate, $\\tau=5$','Free mutation rate, $\\tau=10$',
          'Free mutation rate, $\\tau=100$']
for l, f in zip(labels, allf):
    waic = WAIC_for(f)
    results.append(dict(Model=l, WAIC=waic))
results

[{'Model': 'Without aneuploidy', 'WAIC': -35},
 {'Model': 'Fixed mutation rate, $\\mu=10^{{-{5}}}$, $\\tau=1$', 'WAIC': -16},
 {'Model': 'Fixed mutation rate, $\\mu=10^{{-{6}}}$, $\\tau=1$', 'WAIC': -38},
 {'Model': 'Fixed mutation rate, $\\mu=10^{{-{7}}}$, $\\tau=1$', 'WAIC': -53},
 {'Model': 'Free mutation rate, $\\tau=1$', 'WAIC': 295},
 {'Model': 'Free mutation rate, $\\tau=33/32$', 'WAIC': 266},
 {'Model': 'Free mutation rate, $\\tau=2$', 'WAIC': 501},
 {'Model': 'Free mutation rate, $\\tau=5$', 'WAIC': 376},
 {'Model': 'Free mutation rate, $\\tau=10$', 'WAIC': 318},
 {'Model': 'Free mutation rate, $\\tau=100$', 'WAIC': 319}]

In [116]:
df = pd.DataFrame(results)
df.index = np.arange(1, len(df)+1)
df.index.rename('id', inplace=True)
df = df.reset_index().rename({'index':'index1'}, axis = 'columns')
df.to_csv(f'{output_path}/Table_WAIC.csv', index=False, float_format="%.2f", sep=';')
df

,id,Model,WAIC
0,1,Without aneuploidy,-35
1,2,"Fixed mutation rate, $\mu=10^{{-{5}}}$, $\tau=1$",-16
2,3,"Fixed mutation rate, $\mu=10^{{-{6}}}$, $\tau=1$",-38
3,4,"Fixed mutation rate, $\mu=10^{{-{7}}}$, $\tau=1$",-53
4,5,"Free mutation rate, $\tau=1$",295
5,6,"Free mutation rate, $\tau=33/32$",266
6,7,"Free mutation rate, $\tau=2$",501
7,8,"Free mutation rate, $\tau=5$",376
8,9,"Free mutation rate, $\tau=10$",318
9,10,"Free mutation rate, $\tau=100$",319


In [163]:
WAIC_for(f_prior4, alt_prior=True)

-57

## Analyzing why WAIC for alternative prior is lower

In [141]:
def WAIC_for(f, fixed_mut_rate=False, no_aneuploidy=False, alt_prior=False):
    history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path, f))
    samples, weights = history.get_distribution(m=0, t=None)
    columns = list(samples.columns)
    kde = scipy.stats.gaussian_kde(samples.T.values.tolist(), weights=weights)
    size = 500
    samples = kde.resample(size)
    posteriors = kde.pdf(samples)
    if fixed_mut_rate:
        priors = [prior_fixed_mr.pdf(dict(zip(['p2_tr','p3_w1','p4_w2','p5_w3'],s)))+1e-6 for s in samples.T]
    elif no_aneuploidy:
        priors = [prior_no_aneuploidy.pdf(dict(zip(['p1_mr','p5_w3'],s)))+1e-6 for s in samples.T]
    elif alt_prior:
        priors = [prior_alt.pdf(dict(zip(['p1_mr','p2_tr','p3_w1','p4_w2','p5_w3'],s)))+1e-6 for s in samples.T]
    else:
        priors = [prior.pdf(dict(zip(['p1_mr','p2_tr','p3_w1','p4_w2','p5_w3'],s)))+1e-6 for s in samples.T]
    likelihoods = posteriors/priors 
    return likelihoods
    return round(WAIC_with_weights(likelihoods))

prior4_l = WAIC_for(f_prior4, alt_prior=True)
prior1_l = WAIC_for(k1)

In [148]:
def WAIC_with_weights(likelihoods, rounding=None, weights = None):
    if not weights:
        weights = [1/len(likelihoods)]*len(likelihoods)
    loglik_E = np.log(sum(a*b for a,b in zip(weights,likelihoods)))
    #for p2
    E_loglik = sum(a*np.log(b) for a,b in zip(weights,likelihoods))
    p2 = sum((w*(l-E_loglik)**2 for w,l in zip(weights,np.log(likelihoods))))
    print(-2*loglik_E, 2*p2)
    ans =  -2*loglik_E + 2*p2
    if rounding:
        ans = round(ans, rounding)
    return ans


In [150]:
WAIC_with_weights(prior4_l)

-34.99573235916515 7.1294649888865536


-27.8662673702786

In [156]:
WAIC_with_weights(prior1_l)

-90.69689424435543 355.7377236869039


265.04082944254844

In [153]:
def roundd(v):
    return round(v,3)
        
def findd(n):
    for i in range(0,12):
        m = n*10**i
        if m>=1 and m<=10:
            return i
    raise ValueError(n)
           
# returns (MAP, low_hdi, high_hdi)
def calc_hdi(f, hdi_p=.50):
    history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path, f))
    samples, weights = history.get_distribution(m=0, t=None)
    columns = list(samples.columns)
    kde = scipy.stats.gaussian_kde(samples.T.values.tolist(), weights=weights)
    size = 50000
    samples = kde.resample(size).T
    samples = pd.DataFrame(data=samples)
    MAP = scipy.optimize.minimize(lambda x: -kde.logpdf(x) ,samples.median().values)['x']
    hdi_res = hdi(samples.values, hdi_prob=hdi_p)
    hi = hdi_res.T[1]
    lo = hdi_res.T[0]
    return (MAP, lo, hi)

def print_mode_and_hdi(f, display_cols=['\mu','\delta','w_{2n+1}','w_{2n+1^*}', 'w_{2n^*}'], hdi_p=.50):
    ans = calc_hdi(f,hdi_p)
    orders = [np.array(list(map(findd,ans))) for ans in ans]
    singles = [list(map(roundd,ans*(10**np.array(list(map(findd,ans)))))) for ans in ans]
    for a in list(zip(list(display_cols),*singles, *orders)):
        if a[-1]==0:
            print('${}={} ({}-{})$,'.format(*a))
        else:
            print('${0}={1}\\times10^{{-{4}}} ({2}\\times10^{{-{5}}}-{3}\\times10^{{-{6}}})$,'.format(*a))
    

import warnings
warnings.filterwarnings('ignore')

In [154]:
print_mode_and_hdi(k1)

$\mu=3.014\times10^{-6} (2.272\times10^{-7}-4.175\times10^{-6})$,
$\delta=1.738\times10^{-3} (1.387\times10^{-3}-2.733\times10^{-3})$,
$w_{2n+1}=1.022 (1.021-1.023)$,
$w_{2n+1^*}=1.025 (1.024-1.026)$,
$w_{2n^*}=1.028 (1.026-1.029)$,


In [155]:
print_mode_and_hdi(f_prior4)

$\mu=2.378\times10^{-9} (2.263\times10^{-9}-2.455\times10^{-9})$,
$\delta=2.644\times10^{-3} (2.294\times10^{-3}-2.966\times10^{-3})$,
$w_{2n+1}=1.022 (1.021-1.023)$,
$w_{2n+1^*}=1.032 (1.03-1.035)$,
$w_{2n^*}=1.033 (1.031-1.037)$,
